## Imports

In [22]:
import pandas as pd
import geopandas as gpd
import numpy as np
import folium
import os
import tabula
import googlemaps

## Data read-in

In [2]:
dfs = tabula.read_pdf('118165938811.pdf',pages='all')

In [26]:
gdf = gpd.read_file('nynta2020_23c')

In [27]:
gdf

,BoroCode,BoroName,CountyFIPS,NTA2020,NTAName,NTAAbbrev,NTAType,CDTA2020,CDTAName,Shape_Leng,Shape_Area,geometry
0,3,Brooklyn,047,BK0101,Greenpoint,Grnpt,0,BK01,BK01 Williamsburg-Greenpoint (CD 1 Equivalent),28912.566119,3.532121e+07,"POLYGON ((1003059.997 204572.025, 1002991.367 ..."
1,3,Brooklyn,047,BK0102,Williamsburg,Wllmsbrg,0,BK01,BK01 Williamsburg-Greenpoint (CD 1 Equivalent),28098.026933,2.885431e+07,"POLYGON ((995851.916 203199.332, 995969.192 20..."
2,3,Brooklyn,047,BK0103,South Williamsburg,SWllmsbrg,0,BK01,BK01 Williamsburg-Greenpoint (CD 1 Equivalent),18250.280262,1.520896e+07,"POLYGON ((998047.210 196303.325, 998157.901 19..."
3,3,Brooklyn,047,BK0104,East Williamsburg,EWllmsbrg,0,BK01,BK01 Williamsburg-Greenpoint (CD 1 Equivalent),43184.772815,5.226621e+07,"POLYGON ((1005302.497 199455.730, 1005307.792 ..."
4,3,Brooklyn,047,BK0201,Brooklyn Heights,BkHts,0,BK02,BK02 Downtown Brooklyn-Fort Greene (CD 2 Appro...,14312.504911,9.982322e+06,"POLYGON ((986367.735 190549.239, 985813.835 19..."
...,...,...,...,...,...,...,...,...,...,...,...,...
257,5,Staten Island,085,SI0391,Freshkills Park (South),FrshklPK_S,9,SI03,SI03 South Shore (CD 3 Approximation),33945.420291,4.775877e+07,"POLYGON ((928531.433 150471.714, 929005.649 15..."
258,5,Staten Island,085,SI9561,Fort Wadsworth,FtWdswrth,6,SI95,SI95 Great Kills Park-Fort Wadsworth (JIA 95 A...,14814.414490,9.867249e+06,"POLYGON ((967656.829 155637.131, 967549.629 15..."
259,5,Staten Island,085,SI9591,Hoffman & Swinburne Islands,HffmnIsl,9,SI95,SI95 Great Kills Park-Fort Wadsworth (JIA 95 A...,4743.128127,6.357020e+05,"MULTIPOLYGON (((970217.022 145643.332, 970227...."
260,5,Staten Island,085,SI9592,Miller Field,MllrFld,9,SI95,SI95 Great Kills Park-Fort Wadsworth (JIA 95 A...,19197.200973,1.086680e+07,"POLYGON ((960721.609 147492.642, 960370.525 14..."


In [4]:
cleaned_dfs = [df.replace({r'\r':' '}, regex=True) for df in dfs]

In [5]:
df = pd.concat(cleaned_dfs)

In [8]:
# Remove empty columns
drop_columns = ['Unnamed: 0','Unnamed: 1']
df = df.drop(columns=drop_columns)

# Replace '\r' with single whitespace
df.columns = [col.replace('\r',' ') for col in df.columns]

In [9]:
df = df.dropna(axis=0,subset='Property Address')

In [10]:
df

,No.,Title/Description of Lease,Property Address,Landlord / Counterparty,Landlord / Counterparty Address,Rejection Effective Date,Abandoned Personal Property
1,1.0,Terminated Lease with Signed Termination Agree...,"3000 S Robertson Blvd Los Angeles, CA 90034",3000 S Robertson Property Owner LLC,"4 Park Plaza, Suite 400, Irvine, CA, 92614",11/6/2023,"Miscellaneous Furniture, Fixtures and/or Equip..."
2,2.0,Unexpired Lease,"1 Little W 12th St New York, NY 10014",2 Ninth Avenue Partners LLC,"177 Christopher Street, New York, NY, 10014",11/6/2023,"Miscellaneous Furniture, Fixtures and/or Equip..."
3,3.0,Unexpired Lease,"1 Union Square West New York, NY 10003","Union Square Associates, LLC","One Union Square West, New York, NY, 10003",11/6/2023,"Miscellaneous Furniture, Fixtures and/or Equip..."
4,4.0,Unexpired Lease,"10 East 38th Street New York, NY 10016","10 East 38th Street Company, L.L.C.","34-09 Queens Boulevard, Long Island City, NY, ...",11/6/2023,"Miscellaneous Furniture, Fixtures and/or Equip..."
5,5.0,Unexpired Lease,"10 East 40th Street New York, NY 10016",Ronbet 40th Street LLC,"9 East 40th Street, 8th Floor, New York, NY, 1...",11/6/2023,"Miscellaneous Furniture, Fixtures and/or Equip..."
...,...,...,...,...,...,...,...
3,65.0,"Unexpired Lease For the avoidance of doubt, De...","77 Sands St Brooklyn, NY 11201",RFR/K 77 Sands Owner LLC,"375 Park Avenue, 10th Floor, New York, NY, 10152",11/6/2023,"Miscellaneous Furniture, Fixtures and/or Equip..."
4,66.0,Unexpired License Agreement,"54 W 40th St New York, NY 10018","Blue Bottle Coffee, Inc.","300 Webster Street, Oakland, CA, 94607",11/6/2023,"Miscellaneous Furniture, Fixtures and/or Equip..."
5,67.0,Unexpired Storage License,"311 W 43rd St. New York, NY 10036","DWF V 311 W 43rd, LLC","200 State Street, 12th Floor, Boston, MA, 02109",11/6/2023,"Miscellaneous Furniture, Fixtures and/or Equip..."
6,68.0,Unexpired Sublease,"437 Madison Avenue New York, NY 10022","Santander Holdings USA, Inc.","2 Morrissey Boulevard, Dorchester, MA, 02125",11/6/2023,"Miscellaneous Furniture, Fixtures and/or Equip..."


In [11]:
%store -r google_maps_API_Key
gmaps_key = googlemaps.Client(key=google_maps_API_Key)

In [12]:
def geocode(add):
    g = gmaps_key.geocode(add)
    lat = g[0]["geometry"]["location"]["lat"]
    lng = g[0]["geometry"]["location"]["lng"]
    return (lat, lng)

df['geocoded'] = df['Property Address'].apply(geocode)

In [13]:
df['geocoded'] = df['geocoded'].astype(str)
df[['lat', 'lon']] = df['geocoded'].str.strip('()').str.split(', ', expand=True)
df['lat'] = df['lat'].astype(float)
df['lon'] = df['lon'].astype(float)

In [14]:
df.head(1)

,No.,Title/Description of Lease,Property Address,Landlord / Counterparty,Landlord / Counterparty Address,Rejection Effective Date,Abandoned Personal Property,geocoded,lat,lon
1,1.0,Terminated Lease with Signed Termination Agree...,"3000 S Robertson Blvd Los Angeles, CA 90034",3000 S Robertson Property Owner LLC,"4 Park Plaza, Suite 400, Irvine, CA, 92614",11/6/2023,"Miscellaneous Furniture, Fixtures and/or Equip...","(34.0324217, -118.3893975)",34.032422,-118.389398


In [15]:
df.columns

Index(['No.', 'Title/Description of Lease', 'Property Address',
       'Landlord / Counterparty', 'Landlord / Counterparty Address',
       'Rejection Effective Date', 'Abandoned Personal Property', 'geocoded',
       'lat', 'lon'],
      dtype='object')

In [16]:
def popup_html(row):
    lease_desc = row['Title/Description of Lease']
    prop_address = row['Property Address']
    landlord_or_counterparty = row['Landlord / Counterparty']
    counterparty_address = row['Landlord / Counterparty Address']
    rejection_effective_date = row['Rejection Effective Date']
    abandoned_personal_property = row['Abandoned Personal Property']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Description: </strong>{}'''.format(lease_desc) + '''<br>
    <strong>Property Address: </strong>{}'''.format(prop_address) + '''<br>
    <strong>Landlord or Counterparty: </strong>{}'''.format(landlord_or_counterparty) + '''<br>
    <strong>Counterparty Address: </strong>{}'''.format(counterparty_address) + '''<br>
    <strong>Rejection Effective Date: </strong>{}'''.format(rejection_effective_date) + '''<br>
    <strong>Abandoned Personal Property: </strong>${}'''.format(abandoned_personal_property) + '''<br>
    </html>
    '''
    return html

In [17]:
# Define the bounding box for NYC [min_lon, min_lat, max_lon, max_lat]
nyc_bounding_box = {
    'min_lat': 40.477399,
    'max_lat': 40.917577,
    'min_lon': -74.25909,
    'max_lon': -73.700009
}

# Filter the DataFrame for locations within the bounding box
df_nyc = df[
    (df['lat'] >= nyc_bounding_box['min_lat']) & 
    (df['lat'] <= nyc_bounding_box['max_lat']) & 
    (df['lon'] >= nyc_bounding_box['min_lon']) & 
    (df['lon'] <= nyc_bounding_box['max_lon'])
]

# Now, df_nyc contains only the locations within New York City


In [18]:
df_nyc.info()

<class 'pandas.core.frame.DataFrame'>
Index: 40 entries, 2 to 7
Data columns (total 10 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   No.                              40 non-null     float64
 1   Title/Description of Lease       40 non-null     object 
 2   Property Address                 40 non-null     object 
 3   Landlord / Counterparty          40 non-null     object 
 4   Landlord / Counterparty Address  40 non-null     object 
 5   Rejection Effective Date         40 non-null     object 
 6   Abandoned Personal Property      40 non-null     object 
 7   geocoded                         40 non-null     object 
 8   lat                              40 non-null     float64
 9   lon                              40 non-null     float64
dtypes: float64(3), object(7)
memory usage: 3.4+ KB


In [36]:
import folium
from folium.plugins import MarkerCluster

m = folium.Map(location=df_nyc[["lat", "lon"]].mean().to_list(), zoom_start=10)

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'WeWork Lease Terminations in NYC')


### Create map container ###
m = folium.Map(location=df_nyc[["lat", "lon"]].mean().to_list(),zoom_start=11.5,tiles=None)

for index, row in df_nyc.iterrows():
    marker = folium.Marker(
        location=[row['lat'], row['lon']],
        radius=5,
        fill=True,
        popup=folium.Popup(popup_html(row), max_width=400))
    marker.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)
m.get_root().html.add_child(folium.Element(title_html))

# Display map
m

In [37]:
m.save('index.html')

In [38]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/NYC_WeWork_lease_cuts_Nov_2023
